In [ ]:
!pip install --upgrade keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Dense
import pandas as pd

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/Rohit-Annamaneni-1207/scrap_repo/main/modified_data.csv")
df

,Unnamed: 0,text,label_f,label_h,label_s,clean_comment,Tokens,Length,combined_text
0,2745,sir lovehate relationship ke bare mein aap ka...,1,0,0,sir lovehate relationship ke bare mein aap ka...,"['sir', 'lovehate', 'relationship', 'ke', 'bar...",13,sir लोवेहते relationship के bare mein आप ka क्...
1,2773,same here he is the reason to watch bb but i c...,0,0,0,same here he is the reason to watch bb but i c...,"['same', 'here', 'he', 'is', 'the', 'reason', ...",56,same here he is the reason to watch बब but i c...
2,959,bhai koi iska rape kar doog main bahut khujli ...,0,0,0,bhai koi iska rape kar doog main bahut khujli ...,"['bhai', 'koi', 'iska', 'rape', 'kar', 'doog',...",15,भाई koi इसका rape कर दूग main bahut खुजली ho र...
3,955,buuma raviisha baata karaka pata laga aisa api...,1,0,0,buuma raviisha baata karaka pata laga aisa api...,"['buuma', 'raviisha', 'baata', 'karaka', 'pata...",9,बूम रवीश बात karaka pata लगा ऐसा अपील नाहा
4,4036,gajar mein vitamin c hoty hain jis sey aakhon ...,1,0,0,gajar mein vitamin c hoty hain jis sey aakhon ...,"['gajar', 'mein', 'vitamin', 'c', 'hoty', 'hai...",19,गाजर mein vitamin c होती hain जिस sey आखों की ...
...,...,...,...,...,...,...,...,...,...
10461,1554,bas karo aunty aaplog ka gentle rape ho chuka ...,0,0,0,bas karo aunty aaplog ka gentle rape ho chuka ...,"['bas', 'karo', 'aunty', 'aaplog', 'ka', 'gent...",21,bas karo आंटी आपलोग ka gentle rape ho चूका हैत...
10462,634,masti he mohl me chahi khumari he saarey thak ...,0,0,0,masti he mohl me chahi khumari he saarey thak ...,"['masti', 'he', 'mohl', 'me', 'chahi', 'khumar...",29,मस्ती he महल me चाही खुमारी he सारे थक के beth...
10463,970,dekhna ek din koi inke jindagi ka rape karke c...,0,0,0,dekhna ek din koi inke jindagi ka rape karke c...,"['dekhna', 'ek', 'din', 'koi', 'inke', 'jindag...",11,देखना एक din koi इनके जिंदगी ka rape करके चला ...
10464,2929,aur us waqt kisi ne intolarence hai desh mein ...,0,0,1,aur us waqt kisi ne intolarence hai desh mein ...,"['aur', 'us', 'waqt', 'kisi', 'ne', 'intolaren...",17,और us वक़्त किसी ne इन्टॉलरेंस है देश mein और c...


In [ ]:
X = df["combined_text"].values
X = list(X)
X

In [ ]:
y_hate = df["label_h"].values
np.count_nonzero(y_hate == 1)

1801

In [ ]:
y_fake = df["label_f"].values
np.count_nonzero(y_fake == 1)

6566

In [ ]:
from keras.preprocessing.text import Tokenizer
import numpy as np

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
max_words = 100

tokenizer = Tokenizer(num_words=max_words)

tokenizer.fit_on_texts(X)

sequences = tokenizer.texts_to_sequences(X)

max_sequence_length = 50
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post', truncating='post')

In [ ]:
input_shape = (max_sequence_length,)

inputs = Input(shape=input_shape, name='input_layer')

embedding_size = 100 # replace with the desired embedding size

vocab_size = len(tokenizer.word_index) + 1
embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_size, input_length=max_sequence_length)(inputs)

In [ ]:
embedding_layer

<KerasTensor: shape=(None, 50, 100) dtype=float32 (created by layer 'embedding')>

In [ ]:
padded_sequences

array([[37,  2, 58, ...,  0,  0,  0],
       [40, 67, 11, ...,  0,  0,  0],
       [10, 32,  4, ...,  0,  0,  0],
       ...,
       [28, 32,  7, ...,  0,  0,  0],
       [16, 88, 35, ...,  0,  0,  0],
       [98, 43, 11, ...,  0,  0,  0]], dtype=int32)

In [ ]:
lstm_units = 64
lstm_layer = Bidirectional(LSTM(units=lstm_units, dropout=0.2, recurrent_dropout=0.2))(embedding_layer)

In [ ]:
hate_output = Dense(units=1, activation='sigmoid', name='hate_output')(lstm_layer)

In [ ]:
fake_output = Dense(units=1, activation='sigmoid', name='fake_output')(lstm_layer)

In [ ]:
model = Model(inputs=inputs, outputs=[hate_output, fake_output])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_layer (InputLayer)       [(None, 50)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 50, 100)      2015900     ['input_layer[0][0]']            
                                                                                                  
 bidirectional (Bidirectional)  (None, 128)          84480       ['embedding[0][0]']              
                                                                                                  
 hate_output (Dense)            (None, 1)            129         ['bidirectional[0][0]']          
                                                                                              

In [ ]:
from sklearn.model_selection import train_test_split

X = padded_sequences

X_train, X_test, y_hate_train, y_hate_test, y_fake_train, y_fake_test = train_test_split(X, y_hate, y_fake, test_size=0.2, random_state=42)


print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_hate_train:", y_hate_train.shape)
print("Shape of y_hate_test:", y_hate_test.shape)
print("Shape of y_fake_train:", y_fake_train.shape)
print("Shape of y_fake_test:", y_fake_test.shape)


Shape of X_train: (8372, 50)
Shape of X_test: (2094, 50)
Shape of y1_train: (8372,)
Shape of y1_test: (2094,)
Shape of y2_train: (8372,)
Shape of y2_test: (2094,)


In [ ]:
history = model.fit(X_train, [y_hate_train, y_fake_train], epochs=10, batch_size=32)

Epoch 1/10
262/262 [==============================] - 70s 239ms/step - loss: 0.9978 - hate_output_loss: 0.3843 - fake_output_loss: 0.6136 - hate_output_accuracy: 0.8329 - fake_output_accuracy: 0.6515
Epoch 2/10
262/262 [==============================] - 65s 247ms/step - loss: 0.8971 - hate_output_loss: 0.3406 - fake_output_loss: 0.5565 - hate_output_accuracy: 0.8447 - fake_output_accuracy: 0.6940
Epoch 3/10
262/262 [==============================] - 62s 237ms/step - loss: 0.8855 - hate_output_loss: 0.3357 - fake_output_loss: 0.5498 - hate_output_accuracy: 0.8471 - fake_output_accuracy: 0.7005
Epoch 4/10
262/262 [==============================] - 61s 235ms/step - loss: 0.8772 - hate_output_loss: 0.3312 - fake_output_loss: 0.5460 - hate_output_accuracy: 0.8525 - fake_output_accuracy: 0.7064
Epoch 5/10
262/262 [==============================] - 61s 231ms/step - loss: 0.8682 - hate_output_loss: 0.3271 - fake_output_loss: 0.5410 - hate_output_accuracy: 0.8551 - fake_output_accuracy: 0.7091
